In [1]:
import pandas as pd
import numpy as np
from mvpa2.datasets import dataset_wizard, vstack
from mvpa2.generators.partition import NFoldPartitioner
from mvpa2.measures.base import CrossValidation
from mvpa2.clfs.meta import MappedClassifier
from mvpa2.clfs.svm import LinearCSVMC
    #import mvpa2
    #import mvpa2.datasets
import scipy.io
    #from mvpa2.suite import *
    #Consistent parameters to use for editing datasets
nrois=333
nsess=10
nsub=10
    #make an empty df to store accuracy scores from each run
    #Load task FC
taskFile=scipy.io.loadmat('/Users/Alexis/Box/Quest_Backup/MSC/TaskFC/FC_Parcels/mixed/MSC08_parcel_corrmat.mat')
#Convert to numpy array
task_FC=np.array(taskFile['parcel_corrmat'])
#Replace nans and infs with zero
task_FC=np.nan_to_num(task_FC)
#Index upper triangle of matrix
task_mask=np.triu_indices(nrois,1)
task_ds=np.empty((nsess, int(nrois*(nrois-1)/2)))
task_value=0
        #Loop through all 10 days to reshape correlations into linear form
for sess in range(nsess):
    tmp=task_FC[:,:,sess]
    task_ds[task_value]=tmp[task_mask]
    task_value=task_value+1
        #Load rest
restFile=scipy.io.loadmat('/Users/Alexis/Box/Quest_Backup/MSC/TaskFC/FC_Parcels/rest/MSC08_parcel_corrmat.mat')
rest_FC=np.array(restFile['parcel_corrmat'])
rest_FC=np.nan_to_num(rest_FC)
rest_mask=np.triu_indices(nrois,1)
rest_ds=np.empty((nsess, int(nrois*(nrois-1)/2)))
rest_value=0
for sess in range(nsess):
    tmp=rest_FC[:,:,sess]
    rest_ds[rest_value]=tmp[rest_mask]
    rest_value=rest_value+1
        #Create a training dataset targets 1/0 ==task/rest, chunk=#days
taskdf=dataset_wizard(samples=task_ds, targets=1, chunks=range(10))
restdf=dataset_wizard(samples=rest_ds, targets=0, chunks=range(10))
training=vstack((taskdf, restdf))
        #Linear SVM with cross validation scheme leave one out, output is the accuracy
cv=CrossValidation(LinearCSVMC(), NFoldPartitioner(), 
                      enable_ca=['stats'],
                      errorfx=lambda p, t: np.mean(p == t))
        #Train the classifier         
        #Lets loop through all scenerios for sub and task of interest
testFile=scipy.io.loadmat('/Users/Alexis/Box/Quest_Backup/MSC/TaskFC/FC_Parcels/mixed/MSC09_parcel_corrmat.mat')
test_FC=np.array(testFile['parcel_corrmat'])
test_FC=np.nan_to_num(test_FC)
test_mask=np.triu_indices(nrois,1)
test_ds=np.empty((nsess, int(nrois*(nrois-1)/2)))
test_value=0
for sess in range(nsess):
    tmp=test_FC[:,:,sess]
    test_ds[test_value]=tmp[task_mask]
    test_value=test_value+1
            #and all rest files
test_restFile=scipy.io.loadmat('/Users/Alexis/Box/Quest_Backup/MSC/TaskFC/FC_Parcels/rest/MSC09_parcel_corrmat.mat')
test_rest_FC=np.array(test_restFile['parcel_corrmat'])
test_rest_FC=np.nan_to_num(test_rest_FC)
test_rest_mask=np.triu_indices(nrois,1)
test_rest_ds=np.empty((nsess, int(nrois*(nrois-1)/2)))
test_rest_value=0
for sess in range(nsess):
    tmp=test_rest_FC[:,:,sess]
    test_rest_ds[test_rest_value]=tmp[task_mask]
    test_rest_value=test_rest_value+1
            #Create testing dataset
testing_task=dataset_wizard(samples=test_ds, targets=1, chunks=range(10))
testing_rest=dataset_wizard(samples=test_rest_ds, targets=0, chunks=range(10))
testing=vstack((testing_task, testing_rest))
            #Time to train

In [3]:
pre=cv(testing)
#prediction accuracy
acc=np.mean(pre)

//anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/mvpa2/misc/attrmap.py:153: FutureWarning: Conversion of the second argument of issubdtype from `str` to `str` is deprecated. In future, it will be treated as `np.string_ == np.dtype(str).type`.
  if not np.issubdtype(attr.dtype, str) and not self.mapnumeric:
//anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/mvpa2/clfs/libsvmc/svm.py:215: FutureWarning: Conversion of the second argument of issubdtype from `'c'` to `str` is deprecated. In future, it will be treated as `np.string_ == np.dtype('c').type`.
  if ( np.issubdtype(self.ca.trained_targets.dtype, 'c') or
//anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/mvpa2/clfs/libsvmc/svm.py:216: FutureWarning: Conversion of the second argument of issubdtype from `'U'` to `unicode` is deprecated. In future, it will be treated as `np.unicode_ == np.dtype('U').type`.
  np.issubdtype(self.ca.trained_targets.dtype, 'U') ):


In [8]:
pre

Dataset(array([[0.5],
       [0.5],
       [0.5],
       [0.5],
       [1. ],
       [0.5],
       [0.5],
       [0.5],
       [0.5],
       [0.5]]), sa=SampleAttributesCollection(items=[ArrayCollectable(name='cvfolds', doc=None, value=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), length=10)]), fa=FeatureAttributesCollection(items=[]), a=DatasetAttributesCollection(items=[]))

In [7]:
acc

0.55

In [9]:
cv.train(testing)

In [10]:
msc8=cv(testing)


In [12]:
foo=cv(training)

In [13]:
np.mean(msc8)

0.95

In [14]:
np.mean(foo)

0.55